In [1]:
"|IMPORT PACKAGES|"
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label 
from   bokeh.models       import BoxAnnotation, DatetimeTickFormatter, Span
from   bokeh.transform    import dodge
from   bokeh.models.tools import HoverTool

In [5]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'ANUAL'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B, DQ:DR", skiprows=2, nrows=39, engine='openpyxl') # Be patient...
data = data.dropna()

data["YEAR"] = data["YEAR"].dt.year
data = data.rename(columns={"INTERESES (SIN INTRA-SECTOR PUBLICO)/INGRESOS":"INTERESES1"})
data = data.rename(columns={"INTERESES (CON INTRA-SECTOR PUBLICO)/INGRESOS":"INTERESES2"})

data["INTERESES1"] = data["INTERESES1"]*100
data["INTERESES2"] = data["INTERESES2"]*100

In [6]:
"|CHECK DATA|"
data

,YEAR,INTERESES1,INTERESES2
16,1998,16.501085,16.501085
17,1999,21.290543,21.290543
18,2000,23.741265,23.741265
19,2001,27.377128,27.377128
20,2002,17.009122,17.009122
21,2003,11.332044,11.332044
22,2004,6.823109,6.823109
23,2005,10.038190,10.038190
24,2006,9.291000,9.291000
25,2007,15.016052,15.016052


In [9]:
"|BUILD PLOT|"

data["COLORS"] = ["blue"          , "blue",
                  "red"           , "red" ,
                  "navy"          , "navy",
                  "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue", 
                  "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue", "lightsteelblue",
                  "gold"          , "gold"          , "gold"          , "gold", 
                  "lightsteelblue"]

cds = ColumnDataSource(data)

#BUILD FIGURE
p = figure(title        = "EL HUB ECONÓMICO | INTERESES/INGRESOS TRIBUTARIOS DEL TESORO",
           y_axis_label = "",
           y_range      = (0, 50),
           x_range      = (1998, 2021),
           plot_height  = 400,
           plot_width   = 700)

p.toolbar_location = "right"
p.toolbar.autohide = True
p.add_tools(HoverTool(tooltips = [("Sin intra sector público", "@INTERESES1"), ("Con intra sector pùblico", "@INTERESES2")]))


#AXIS 1 (LEFT)
p.vbar(x=dodge('YEAR', -0.20, range=p.x_range), top='INTERESES1', fill_color="COLORS", width=0.4, fill_alpha=0.50, legend_label="Sin intereses intra sector publico", muted_alpha=0.2, source=cds)
p.vbar(x=dodge('YEAR',  0.20, range=p.x_range), top='INTERESES2', fill_color="COLORS", width=0.4, fill_alpha=0.50, legend_label="Con inteteses intra sector publico", muted_alpha=0.2, source=cds,
       hatch_pattern="/", hatch_alpha=0.25)


#LEGEND
p.legend.location     = "top_left"
p.legend.orientation  = "horizontal"
p.legend.click_policy = "mute"
show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="intereses_ingresos.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="intereses_ingresos.html", title="Deuda pública")
save(p)

In [10]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "intereses_ingresos"))
with open("intereses_ingresos.json", "w") as fp:
    json.dump(j, fp)